<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in New York City</font></h1>

## Introduction

In this lab, you will learn how to convert addresses into their equivalent latitude and longitude values. Also, you will use the Foursquare API to explore neighborhoods in New York City. You will use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. You will use the *k*-means clustering algorithm to complete this task. Finally, you will use the Folium library to visualize the neighborhoods in New York City and their emerging clusters.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in New York City</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## 1. Download and Explore Dataset

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 

Luckily, this dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

For your convenience, I downloaded the files and placed it on the server, so you can simply run a `wget` command and access the data. So let's go ahead and do that.

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


#### Load and explore the data

Next, let's load the data.

In [4]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Let's take a quick look at the data.

In [21]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

Notice how all the relevant data is in the *features* key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [5]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list.

In [12]:
neighborhoods_data

[{'type': 'Feature',
  'id': 'nyu_2451_34572.1',
  'geometry': {'type': 'Point',
   'coordinates': [-73.84720052054902, 40.89470517661]},
  'geometry_name': 'geom',
  'properties': {'name': 'Wakefield',
   'stacked': 1,
   'annoline1': 'Wakefield',
   'annoline2': None,
   'annoline3': None,
   'annoangle': 0.0,
   'borough': 'Bronx',
   'bbox': [-73.84720052054902,
    40.89470517661,
    -73.84720052054902,
    40.89470517661]}},
 {'type': 'Feature',
  'id': 'nyu_2451_34572.2',
  'geometry': {'type': 'Point',
   'coordinates': [-73.82993910812398, 40.87429419303012]},
  'geometry_name': 'geom',
  'properties': {'name': 'Co-op City',
   'stacked': 2,
   'annoline1': 'Co-op',
   'annoline2': 'City',
   'annoline3': None,
   'annoangle': 0.0,
   'borough': 'Bronx',
   'bbox': [-73.82993910812398,
    40.87429419303012,
    -73.82993910812398,
    40.87429419303012]}},
 {'type': 'Feature',
  'id': 'nyu_2451_34572.3',
  'geometry': {'type': 'Point',
   'coordinates': [-73.82780644716412, 

#### Tranform the data into a *pandas* dataframe

The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe. So let's start by creating an empty dataframe.

In [7]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [8]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.

In [9]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.

In [11]:
neighborhoods.head(100)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [28]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [29]:
print(len(neighborhoods))

306


#### Use geopy library to get the latitude and longitude values of New York City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [30]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of New York with neighborhoods superimposed on top.

In [32]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.

In [33]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Let's get the geographical coordinates of Manhattan.

In [34]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


As we did with all of New York City, let's visualizat Manhattan the neighborhoods in it.

In [35]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [36]:
CLIENT_ID = 'X5OBM0WZCSVVJASBUED5CUOTIGF3D4UKUTXLQ3G44JTG3RLU' # your Foursquare ID
CLIENT_SECRET = '4OMKQ2OOFVOV1BC1ZMQAHN0QZAPUICT3SOUAI2TKDMF4ZEBB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: X5OBM0WZCSVVJASBUED5CUOTIGF3D4UKUTXLQ3G44JTG3RLU
CLIENT_SECRET:4OMKQ2OOFVOV1BC1ZMQAHN0QZAPUICT3SOUAI2TKDMF4ZEBB


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [37]:
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

Get the neighborhood's latitude and longitude values.

In [38]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

In [39]:
len(manhattan_data)

40

First, let's create the GET request URL. Name your URL **url**.

In [40]:
# type your answer here
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url




'https://api.foursquare.com/v2/venues/explore?&client_id=X5OBM0WZCSVVJASBUED5CUOTIGF3D4UKUTXLQ3G44JTG3RLU&client_secret=4OMKQ2OOFVOV1BC1ZMQAHN0QZAPUICT3SOUAI2TKDMF4ZEBB&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

Double-click __here__ for the solution.
<!-- The correct answer is:
LIMIT = 100 # limit of number of venues returned by Foursquare API
-->

<!--
radius = 500 # define radius
-->

<!--
\\ # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
--> 

Send the GET request and examine the resutls

In [41]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9c5343c546f3001ba3115e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 25,
  'suggestedBounds': {'ne': {'lat': 40.88105078329964,
    'lng': -73.90471933917806},
   'sw': {'lat': 40.87205077429964, 'lng': -73.91659997808156}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b4429abf964a52037f225e3',
       'name': "Arturo's",
       'location': {'address': '5198 Broadway',
        'crossStreet': 'at 225th St.',
        'lat': 40.87441177110231,
        'lng': -73.91027100981574,
        'labeledLatLngs': [{'label'

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [112]:
#results['response']['groups'][0]['items']
results['response']['groups'][0]

{'type': 'Recommended Places',
 'name': 'recommended',
 'items': [{'reasons': {'count': 0,
    'items': [{'summary': 'This spot is popular',
      'type': 'general',
      'reasonName': 'globalInteractionReason'}]},
   'venue': {'id': '4b4429abf964a52037f225e3',
    'name': "Arturo's",
    'location': {'address': '5198 Broadway',
     'crossStreet': 'at 225th St.',
     'lat': 40.87441177110231,
     'lng': -73.91027100981574,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.87441177110231,
       'lng': -73.91027100981574},
      {'label': '?', 'lat': 40.874401, 'lng': -73.910339}],
     'distance': 240,
     'postalCode': '10463',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['5198 Broadway (at 225th St.)',
      'New York, NY 10463',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'P

In [80]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        print(categories_list[0]['name'])
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [81]:
venues = results['response']['groups'][0]['items']
#print(venues)
    
nearby_venues = json_normalize(venues) # flatten JSON
#nearby_venues
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
#nearby_venues
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

Pizza Place
Yoga Studio
Diner
Donut Shop
Coffee Shop
Gym
Gym
Department Store
Seafood Restaurant
Tennis Stadium
Pharmacy
Coffee Shop
Discount Store
Supplement Shop
Ice Cream Shop
Sandwich Place
Miscellaneous Shop
American Restaurant
Video Game Store
Sandwich Place
Bank
Steakhouse
Kids Store
Sandwich Place
Deli / Bodega


/home/apu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Dunkin',Donut Shop,40.877136,-73.906666
4,Starbucks,Coffee Shop,40.877531,-73.905582


And how many venues were returned by Foursquare?

In [82]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

25 venues were returned by Foursquare.


<a id='item2'></a>

## 2. Explore Neighborhoods in Manhattan

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [91]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues, venues_list)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [92]:
# type your answer here
(manhattan_venues, venues_list) = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )


Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


Double-click __here__ for the solution.
<!-- The correct answer is:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
--> 

In [107]:
venues_list[5]

[('Manhattanville',
  40.8169344294978,
  -73.9573853935188,
  'Oasis Juice Bar',
  40.815016957500134,
  -73.95887946970998,
  'Juice Bar'),
 ('Manhattanville',
  40.8169344294978,
  -73.9573853935188,
  'Jin Ramen',
  40.815406172563904,
  -73.95854739270733,
  'Ramen Restaurant'),
 ('Manhattanville',
  40.8169344294978,
  -73.9573853935188,
  'Steep Rock West',
  40.81666795087888,
  -73.95796911357189,
  'Climbing Gym'),
 ('Manhattanville',
  40.8169344294978,
  -73.9573853935188,
  'Dinosaur Bar-B-Que',
  40.818146873796856,
  -73.9606735523417,
  'BBQ Joint'),
 ('Manhattanville',
  40.8169344294978,
  -73.9573853935188,
  'Kissaten Jin',
  40.81545749902605,
  -73.9584688976533,
  'Café'),
 ('Manhattanville',
  40.8169344294978,
  -73.9573853935188,
  'Go! Go! Curry!',
  40.81547267978402,
  -73.95823441394805,
  'Japanese Curry Restaurant'),
 ('Manhattanville',
  40.8169344294978,
  -73.9573853935188,
  'West Harlem Piers Park',
  40.81879256812002,
  -73.96102935944113,
  'Park

#### Let's check the size of the resulting dataframe

In [111]:
#[item for venue_list in venues_list for item in venue_list]

#print('\n\n\n')
for venue_list in venues_list:
    #print(venue_list)
    for item in venue_list:
        print(item)
    

('Marble Hill', 40.87655077879964, -73.91065965862981, "Arturo's", 40.87441177110231, -73.91027100981574, 'Pizza Place')
('Marble Hill', 40.87655077879964, -73.91065965862981, 'Bikram Yoga', 40.876843690797934, -73.90620384419528, 'Yoga Studio')
('Marble Hill', 40.87655077879964, -73.91065965862981, 'Tibbett Diner', 40.8804044222466, -73.90893738006402, 'Diner')
('Marble Hill', 40.87655077879964, -73.91065965862981, "Dunkin'", 40.87713584201589, -73.90666550701411, 'Donut Shop')
('Marble Hill', 40.87655077879964, -73.91065965862981, 'Starbucks', 40.87753134921497, -73.90558216359267, 'Coffee Shop')
('Marble Hill', 40.87655077879964, -73.91065965862981, 'Astral Fitness & Wellness Center', 40.87670526507199, -73.90637207670373, 'Gym')
('Marble Hill', 40.87655077879964, -73.91065965862981, 'Blink Fitness', 40.877271495944626, -73.90559491338075, 'Gym')
('Marble Hill', 40.87655077879964, -73.91065965862981, 'T.J. Maxx', 40.87723198343352, -73.90504239962168, 'Department Store')
('Marble Hi

('Manhattanville', 40.8169344294978, -73.9573853935188, 'Hudson River Greenway Bike Path', 40.819281875389116, -73.96073341369629, 'Bike Trail')
('Manhattanville', 40.8169344294978, -73.9573853935188, 'The Expat', 40.815355, -73.960178, 'Gastropub')
('Manhattanville', 40.8169344294978, -73.9573853935188, 'Pisticci Ristorante', 40.81401515598367, -73.9602659078139, 'Italian Restaurant')
('Manhattanville', 40.8169344294978, -73.9573853935188, 'Fairway Market', 40.81999621370494, -73.95973825129128, 'Supermarket')
('Manhattanville', 40.8169344294978, -73.9573853935188, 'Chapati House - NYC', 40.81457249971737, -73.95915381671377, 'Indian Restaurant')
('Manhattanville', 40.8169344294978, -73.9573853935188, 'Plowshares Coffee', 40.813705, -73.955776, 'Coffee Shop')
('Manhattanville', 40.8169344294978, -73.9573853935188, 'El Puerto Seafood and Fish Market', 40.813861024837166, -73.95673661274444, 'Seafood Restaurant')
('Manhattanville', 40.8169344294978, -73.9573853935188, 'The World Famous 

('Lenox Hill', 40.76811265828733, -73.9588596881376, 'Mission Ceviche', 40.769218, -73.957901, 'Restaurant')
('Lenox Hill', 40.76811265828733, -73.9588596881376, 'Marymount Manhattan College', 40.769232464055726, -73.95965795209456, 'College Academic Building')
('Lenox Hill', 40.76811265828733, -73.9588596881376, 'Anthropologie', 40.76929584181941, -73.96108483574552, "Women's Store")
('Lenox Hill', 40.76811265828733, -73.9588596881376, 'Donna Margherita', 40.766452, -73.959905, 'Pizza Place')
('Lenox Hill', 40.76811265828733, -73.9588596881376, 'A Matter of Health', 40.76802836725529, -73.95593309596553, 'Health Food Store')
('Lenox Hill', 40.76811265828733, -73.9588596881376, 'La Esquina', 40.76945123405223, -73.95778164213363, 'Taco Place')
('Lenox Hill', 40.76811265828733, -73.9588596881376, 'Bohemian Spirit Restaurant', 40.7691142, -73.9565393, 'Czech Restaurant')
('Lenox Hill', 40.76811265828733, -73.9588596881376, 'Equinox East 74th Street', 40.77046, -73.957387, 'Gym')
('Lenox 

('Clinton', 40.75910089146212, -73.99611936309479, 'UCB Theatre Hell’s Kitchen', 40.760499950120476, -73.997863306895, 'Comedy Club')
('Clinton', 40.75910089146212, -73.99611936309479, 'Manhattan Plaza Health Club (MPHC)', 40.75996689999999, -73.99458489999999, 'Gym / Fitness Center')
('Clinton', 40.75910089146212, -73.99611936309479, 'The Victory', 40.75927433585485, -73.99646791200374, 'Building')
('Clinton', 40.75910089146212, -73.99611936309479, 'Mémé Mediterranean', 40.760904754375375, -73.99460052237751, 'Mediterranean Restaurant')
('Clinton', 40.75910089146212, -73.99611936309479, 'Rustic Table', 40.759754, -73.996013, 'Café')
('Clinton', 40.75910089146212, -73.99611936309479, 'Equinox® at M Club', 40.75920115265559, -73.99508032030828, 'Gym / Fitness Center')
('Clinton', 40.75910089146212, -73.99611936309479, 'Pan Aqua Diving', 40.7599386094597, -73.99452003010043, 'Sporting Goods Shop')
('Clinton', 40.75910089146212, -73.99611936309479, 'Stage 42', 40.758866974859906, -73.9938

('Chelsea', 40.744034706747975, -74.00311633472813, 'Juice Press', 40.74371218067827, -74.00705280870267, 'Juice Bar')
('Greenwich Village', 40.72693288536128, -73.99991402945902, 'Chobani', 40.72581867292355, -74.00098495024535, 'Snack Place')
('Greenwich Village', 40.72693288536128, -73.99991402945902, 'Carbone', 40.72790327156426, -74.00013635911864, 'Italian Restaurant')
('Greenwich Village', 40.72693288536128, -73.99991402945902, 'Rapha NYC', 40.72596825325445, -74.00122621589483, 'Clothing Store')
('Greenwich Village', 40.72693288536128, -73.99991402945902, 'Local', 40.727089082727915, -74.00191220344334, 'Beer Bar')
('Greenwich Village', 40.72693288536128, -73.99991402945902, 'Terra Blues', 40.72828843382252, -73.99938089456808, 'Jazz Club')
('Greenwich Village', 40.72693288536128, -73.99991402945902, 'Banter', 40.7279592919172, -74.00120508994551, 'Café')
('Greenwich Village', 40.72693288536128, -73.99991402945902, 'Sacred Sounds Yoga', 40.72863771916715, -74.00011526297406, 'Y

('Tribeca', 40.721521967443216, -74.01068328559087, 'Stuyvesant High School', 40.717608879529976, -74.01320281198944, 'High School')
('Tribeca', 40.721521967443216, -74.01068328559087, 'Tribeca Wines', 40.717697481898696, -74.01060959763295, 'Wine Shop')
('Tribeca', 40.721521967443216, -74.01068328559087, 'Taste of Tribeca', 40.717330858430344, -74.00953604057173, 'Food Court')
('Tribeca', 40.721521967443216, -74.01068328559087, 'Basketball Courts @ Hudson River Park', 40.72562447832109, -74.01153681738498, 'Basketball Court')
('Tribeca', 40.721521967443216, -74.01068328559087, 'Tribeca Skate Park', 40.719432436121856, -74.01270358433756, 'Skate Park')
('Tribeca', 40.721521967443216, -74.01068328559087, 'U.S.C.G. Cutter Lilac', 40.71923228353776, -74.0124281478333, 'Boat or Ferry')
('Little Italy', 40.71932379395907, -73.99730467208073, 'La Compagnie des Vins Surnaturels', 40.720447943059405, -73.9979693380989, 'Wine Bar')
('Little Italy', 40.71932379395907, -73.99730467208073, 'sweetg

('Gramercy', 40.737209832715, -73.98137594833541, 'Paddy Maguires Ale House', 40.73675606009451, -73.98465890166635, 'Pub')
('Gramercy', 40.737209832715, -73.98137594833541, 'Gramercy Park Bar', 40.73525223152769, -73.98284135652284, 'Bar')
('Gramercy', 40.737209832715, -73.98137594833541, "Bruno's Gourmet Market", 40.73304965455923, -73.98128793943675, 'Sandwich Place')
('Gramercy', 40.737209832715, -73.98137594833541, 'Chipotle Mexican Grill', 40.7329583, -73.9808002, 'Mexican Restaurant')
('Gramercy', 40.737209832715, -73.98137594833541, 'Mattress Firm Stuy Town', 40.73466716773827, -73.98056763864042, 'Mattress Store')
('Gramercy', 40.737209832715, -73.98137594833541, 'Walgreens', 40.73322633791861, -73.98068904876709, 'Pharmacy')
('Gramercy', 40.737209832715, -73.98137594833541, 'Train Playground', 40.73299627604286, -73.98054289555807, 'Playground')
('Gramercy', 40.737209832715, -73.98137594833541, 'Starbucks', 40.73323, -73.98082, 'Coffee Shop')
('Battery Park City', 40.71193198

('Noho', 40.72325901885768, -73.98843368023597, 'Supper', 40.72291486887997, -73.9853185415268, 'Italian Restaurant')
('Noho', 40.72325901885768, -73.98843368023597, 'Emmy Squared', 40.72586676457121, -73.98663786068168, 'Pizza Place')
('Noho', 40.72325901885768, -73.98843368023597, 'Rawsome Treats', 40.720888, -73.988547, 'Dessert Shop')
('Noho', 40.72325901885768, -73.98843368023597, 'Y7 East Village', 40.72186898265034, -73.98508543421316, 'Yoga Studio')
('Noho', 40.72325901885768, -73.98843368023597, 'Jadis', 40.72093779022683, -73.99107143734521, 'Wine Bar')
('Noho', 40.72325901885768, -73.98843368023597, 'Double Zero', 40.72610826388715, -73.98980624666206, 'Pizza Place')
('Noho', 40.72325901885768, -73.98843368023597, 'Upstate Craft Beer and Oyster Bar', 40.726331, -73.986453, 'Seafood Restaurant')
('Noho', 40.72325901885768, -73.98843368023597, 'Lancelotti Housewares', 40.724545, -73.98444, 'Furniture / Home Store')
('Noho', 40.72325901885768, -73.98843368023597, 'New York Pila

('Sutton Place', 40.76028033131374, -73.96355614094303, 'Bumble & Bumble', 40.75990252535224, -73.96910482861064, 'Salon / Barbershop')
('Sutton Place', 40.76028033131374, -73.96355614094303, 'juice press', 40.7634671312426, -73.96512169599512, 'Vegetarian / Vegan Restaurant')
('Sutton Place', 40.76028033131374, -73.96355614094303, 'Felice 64', 40.762706137157465, -73.95973879585064, 'Wine Bar')
('Sutton Place', 40.76028033131374, -73.96355614094303, 'The Greats of Craft', 40.7563665, -73.96466179999999, 'Beer Bar')
('Sutton Place', 40.76028033131374, -73.96355614094303, 'Ravagh Persian Grill', 40.76178422133652, -73.96044293856625, 'Persian Restaurant')
('Sutton Place', 40.76028033131374, -73.96355614094303, 'Sprinkles Cupcakes', 40.76317085988467, -73.967511235118, 'Cupcake Shop')
('Sutton Place', 40.76028033131374, -73.96355614094303, 'Sutton Inn', 40.75665315495737, -73.96484669560901, 'American Restaurant')
('Sutton Place', 40.76028033131374, -73.96355614094303, 'Angkor Cambodian 

('Stuyvesant Town', 40.73099955477061, -73.97405170469203, 'Solar One', 40.73444177328803, -73.97434969023425, 'Park')
('Stuyvesant Town', 40.73099955477061, -73.97405170469203, 'BP', 40.73510670723425, -73.97455404113494, 'Gas Station')
('Stuyvesant Town', 40.73099955477061, -73.97405170469203, 'Zum Kaboom', 40.7348, -73.974526, 'German Restaurant')
('Stuyvesant Town', 40.73099955477061, -73.97405170469203, 'The Paddle Wheel Queen', 40.73524250780014, -73.97332906723022, 'Boat or Ferry')
('Stuyvesant Town', 40.73099955477061, -73.97405170469203, 'Marco Polo Cruises - Rendezvouz', 40.73547013604506, -73.97389233112335, 'Boat or Ferry')
('Flatiron', 40.739673047638426, -73.9909471052826, 'The Fhitting Room', 40.739856864845336, -73.99319384310543, 'Gym')
('Flatiron', 40.739673047638426, -73.9909471052826, 'Fishs Eddy', 40.738605357398285, -73.99001680796049, 'Furniture / Home Store')
('Flatiron', 40.739673047638426, -73.9909471052826, 'sweetgreen', 40.73806133544267, -73.99122334551976,

In [115]:
print(manhattan_venues.shape)
manhattan_venues.head(100)

(3012, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.910660,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.910660,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.910660,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.910660,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.910660,Starbucks,40.877531,-73.905582,Coffee Shop
5,Marble Hill,40.876551,-73.910660,Astral Fitness & Wellness Center,40.876705,-73.906372,Gym
6,Marble Hill,40.876551,-73.910660,Blink Fitness,40.877271,-73.905595,Gym
7,Marble Hill,40.876551,-73.910660,T.J. Maxx,40.877232,-73.905042,Department Store
8,Marble Hill,40.876551,-73.910660,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant
9,Marble Hill,40.876551,-73.910660,TCR The Club of Riverdale,40.878628,-73.914568,Tennis Stadium


Let's check how many venues were returned for each neighborhood

In [116]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,59,59,59,59,59,59
Carnegie Hill,86,86,86,86,86,86
Central Harlem,45,45,45,45,45,45
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,90,90,90,90,90,90
Clinton,100,100,100,100,100,100
East Harlem,41,41,41,41,41,41
East Village,100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [117]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 319 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [120]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head(20)

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting

And let's examine the new dataframe size.

In [121]:
manhattan_onehot.shape

(3012, 320)

In [126]:
a=[1,2,3,4,5]
a[:-1]

[1, 2, 3, 4]

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [127]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting

#### Let's confirm the new size

In [128]:
manhattan_grouped.shape

(40, 320)

#### Let's print each neighborhood along with the top 5 most common venues

In [131]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    #print(temp)
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0           Park  0.10
1          Hotel  0.08
2            Gym  0.07
3  Memorial Site  0.05
4          Plaza  0.03


----Carnegie Hill----
                 venue  freq
0          Coffee Shop  0.09
1                 Café  0.05
2  Japanese Restaurant  0.03
3               Bakery  0.03
4            Wine Shop  0.03


----Central Harlem----
                  venue  freq
0    African Restaurant  0.04
1   American Restaurant  0.04
2                   Bar  0.04
3         Deli / Bodega  0.04
4  Gym / Fitness Center  0.04


----Chelsea----
                 venue  freq
0          Art Gallery  0.13
1          Coffee Shop  0.07
2  American Restaurant  0.03
3   Italian Restaurant  0.03
4       Ice Cream Shop  0.03


----Chinatown----
                venue  freq
0  Chinese Restaurant  0.08
1              Bakery  0.05
2        Cocktail Bar  0.04
3        Optical Shop  0.03
4                 Spa  0.03


----Civic Center----
               venue  freq
0  

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [133]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [134]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Gym,Memorial Site,Italian Restaurant,Food Court,Coffee Shop,Burger Joint,Gourmet Shop,Plaza
1,Carnegie Hill,Coffee Shop,Café,Yoga Studio,Pizza Place,Bakery,Bar,Bookstore,Gym,Gym / Fitness Center,Italian Restaurant
2,Central Harlem,Bar,Deli / Bodega,African Restaurant,American Restaurant,Gym / Fitness Center,Chinese Restaurant,French Restaurant,Seafood Restaurant,Dessert Shop,Bookstore
3,Chelsea,Art Gallery,Coffee Shop,American Restaurant,Ice Cream Shop,Italian Restaurant,Café,Cupcake Shop,Park,Pizza Place,Bakery
4,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,Spa,American Restaurant,Optical Shop,Salon / Barbershop,Coffee Shop,Dumpling Restaurant,Shanghai Restaurant


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [135]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

manhattan_grouped_clustering

,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,S

In [136]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 0, 1, 1, 1, 3, 0, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [138]:
manhattan_data

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688
5,Manhattan,Manhattanville,40.816934,-73.957385
6,Manhattan,Central Harlem,40.815976,-73.943211
7,Manhattan,East Harlem,40.792249,-73.944182
8,Manhattan,Upper East Side,40.775639,-73.960508
9,Manhattan,Yorkville,40.775930,-73.947118


In [137]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Deli / Bodega,Supplement Shop,Steakhouse,Seafood Restaurant,Pizza Place,Department Store
1,Manhattan,Chinatown,40.715618,-73.994279,1,Chinese Restaurant,Bakery,Cocktail Bar,Spa,American Restaurant,Optical Shop,Salon / Barbershop,Coffee Shop,Dumpling Restaurant,Shanghai Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,3,Café,Bakery,Chinese Restaurant,Mexican Restaurant,Mobile Phone Shop,Grocery Store,Supermarket,Bank,New American Restaurant,Supplement Shop
3,Manhattan,Inwood,40.867684,-73.921210,3,Café,Mexican Restaurant,Bakery,Pizza Place,Restaurant,Lounge,Chinese Restaurant,Wine Bar,American Restaurant,Deli / Bodega
4,Manhattan,Hamilton Heights,40.823604,-73.949688,3,Pizza Place,Coffee Shop,Café,Mexican Restaurant,Deli / Bodega,Cocktail Bar,Indian Restaurant,Sushi Restaurant,Park,Yoga Studio


Finally, let's visualize the resulting clusters

In [139]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [140]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Manhattanville,Coffee Shop,Seafood Restaurant,Mexican Restaurant,Park,Deli / Bodega,Italian Restaurant,Spanish Restaurant,Bus Station,Bus Stop,Plaza
8,Upper East Side,Italian Restaurant,Gym / Fitness Center,Bakery,Exhibit,Coffee Shop,Yoga Studio,Pizza Place,Spa,French Restaurant,Juice Bar
9,Yorkville,Italian Restaurant,Coffee Shop,Gym,Bar,Deli / Bodega,Japanese Restaurant,Wine Shop,Diner,Pizza Place,Mexican Restaurant
10,Lenox Hill,Italian Restaurant,Coffee Shop,Pizza Place,Sushi Restaurant,Cocktail Bar,Gym,Gym / Fitness Center,Burger Joint,Café,Cycle Studio
12,Upper West Side,Italian Restaurant,Bakery,Bar,Coffee Shop,Seafood Restaurant,Middle Eastern Restaurant,Bagel Shop,Thai Restaurant,Vegetarian / Vegan Restaurant,Mediterranean Restaurant
17,Chelsea,Art Gallery,Coffee Shop,American Restaurant,Ice Cream Shop,Italian Restaurant,Café,Cupcake Shop,Park,Pizza Place,Bakery
18,Greenwich Village,Italian Restaurant,Café,Coffee Shop,Gym,Dessert Shop,Chinese Restaurant,Indie Movie Theater,Sushi Restaurant,Jazz Club,Gourmet Shop
19,East Village,Pizza Place,Bar,Coffee Shop,Cocktail Bar,Japanese Restaurant,Juice Bar,Greek Restaurant,Grocery Store,Salon / Barbershop,Italian Restaurant
22,Little Italy,Chinese Restaurant,Italian Restaurant,Spa,Bubble Tea Shop,Mediterranean Restaurant,Café,Bakery,Thai Restaurant,Cosmetics Shop,Hotel
23,Soho,Italian Restaurant,Coffee Shop,Café,Mediterranean Restaurant,Bakery,Clothing Store,Dessert Shop,Salon / Barbershop,Art Gallery,Gym


#### Cluster 2

In [141]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Deli / Bodega,Supplement Shop,Steakhouse,Seafood Restaurant,Pizza Place,Department Store
1,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,Spa,American Restaurant,Optical Shop,Salon / Barbershop,Coffee Shop,Dumpling Restaurant,Shanghai Restaurant
6,Central Harlem,Bar,Deli / Bodega,African Restaurant,American Restaurant,Gym / Fitness Center,Chinese Restaurant,French Restaurant,Seafood Restaurant,Dessert Shop,Bookstore
13,Lincoln Square,Gym / Fitness Center,Plaza,Café,Italian Restaurant,Performing Arts Venue,Theater,Concert Hall,Indie Movie Theater,Wine Shop,American Restaurant
14,Clinton,Theater,Gym / Fitness Center,Coffee Shop,Gym,Spa,Thai Restaurant,Italian Restaurant,Sandwich Place,Pizza Place,Hotel
15,Midtown,Hotel,Coffee Shop,Clothing Store,Theater,Japanese Restaurant,Pizza Place,Mediterranean Restaurant,Gym,Spa,Bakery
16,Murray Hill,Sandwich Place,Hotel,Bar,Pizza Place,Burger Joint,Gym / Fitness Center,Deli / Bodega,Steakhouse,Sushi Restaurant,Mediterranean Restaurant
27,Gramercy,Coffee Shop,Pizza Place,Italian Restaurant,Cocktail Bar,Bar,Playground,Grocery Store,Ice Cream Shop,American Restaurant,Spa
28,Battery Park City,Park,Hotel,Gym,Memorial Site,Italian Restaurant,Food Court,Coffee Shop,Burger Joint,Gourmet Shop,Plaza
29,Financial District,Coffee Shop,Hotel,Japanese Restaurant,Cocktail Bar,Salad Place,Pizza Place,Gym / Fitness Center,Falafel Restaurant,Gym,American Restaurant


#### Cluster 3

In [142]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Park,Boat or Ferry,Bar,Farmers Market,Gym / Fitness Center,Baseball Field,Harbor / Marina,Pet Service,Cocktail Bar,Heliport


#### Cluster 4

In [143]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Café,Bakery,Chinese Restaurant,Mexican Restaurant,Mobile Phone Shop,Grocery Store,Supermarket,Bank,New American Restaurant,Supplement Shop
3,Inwood,Café,Mexican Restaurant,Bakery,Pizza Place,Restaurant,Lounge,Chinese Restaurant,Wine Bar,American Restaurant,Deli / Bodega
4,Hamilton Heights,Pizza Place,Coffee Shop,Café,Mexican Restaurant,Deli / Bodega,Cocktail Bar,Indian Restaurant,Sushi Restaurant,Park,Yoga Studio
7,East Harlem,Mexican Restaurant,Bakery,Thai Restaurant,Deli / Bodega,Latin American Restaurant,Gym,Beer Bar,Cocktail Bar,Liquor Store,Steakhouse
11,Roosevelt Island,Park,Japanese Restaurant,Outdoors & Recreation,Greek Restaurant,Metro Station,Supermarket,Bubble Tea Shop,Bus Line,Farmers Market,Soccer Field
20,Lower East Side,Chinese Restaurant,Art Gallery,Pizza Place,Bakery,Theater,Café,Park,Mexican Restaurant,Women's Store,Juice Bar
21,Tribeca,Italian Restaurant,Park,Wine Bar,Spa,Café,Wine Shop,Hotel,Bakery,Scenic Lookout,Greek Restaurant
25,Manhattan Valley,Yoga Studio,Mexican Restaurant,Pizza Place,Bar,Thai Restaurant,Korean Restaurant,Indian Restaurant,Szechuan Restaurant,Bubble Tea Shop,Café
26,Morningside Heights,Park,Bookstore,American Restaurant,Coffee Shop,Deli / Bodega,Burger Joint,Tennis Court,Ice Cream Shop,Farmers Market,Mexican Restaurant
35,Turtle Bay,Sushi Restaurant,Italian Restaurant,Deli / Bodega,Café,Park,Hotel,Coffee Shop,Wine Bar,French Restaurant,Japanese Restaurant


#### Cluster 5

In [144]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Midtown South,Korean Restaurant,Hotel,Japanese Restaurant,Dessert Shop,Hotel Bar,Burger Joint,Café,Gym / Fitness Center,New American Restaurant,Lounge


### Thank you for completing this lab!

This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson/) and [Polong Lin](https://www.linkedin.com/in/polonglin/). I hope you found this lab interesting and educational. Feel free to contact us if you have any questions!

This notebook is part of a course on **Coursera** called *Applied Data Science Capstone*. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week3_LAB2).

<hr>

Copyright &copy; 2018 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).